In [2]:
!pip install scrapy
!pip install bs4

    100% |████████████████████████████████| 256kB 1.8MB/s ta 0:00:01
    100% |████████████████████████████████| 5.9MB 113kB/s eta 0:00:01
    100% |████████████████████████████████| 3.0MB 218kB/s eta 0:00:01
    100% |████████████████████████████████| 71kB 5.8MB/s eta 0:00:01
    100% |████████████████████████████████| 81kB 6.5MB/s eta 0:00:01
    100% |████████████████████████████████| 153kB 4.2MB/s eta 0:00:01
  Running setup.py bdist_wheel for PyDispatcher ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/88/99/96/cfef6665f9cb1522ee6757ae5955feedf2fe25f1737f91fa7f
  Running setup.py bdist_wheel for Twisted ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/b3/76/f7/85353c829c0881e74b5366ce0ed59042b098bb4903e2da8828
  Running setup.py bdist_wheel for zope.interface ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/c6/b2/d2/be6785a207eaa58d76debc10c9d5c66196b40a88abb61d6af7
Successfully built PyDispatcher Twisted zope.interface
You are using pip 

In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess
from bs4 import BeautifulSoup
import os
import requests
from datetime import datetime, timedelta

In [2]:
RU_MONTH_VALUES = {
    'января': '01',
    'февраля': '02',
    'марта': '03',
    'апреля': '04',
    'мая': '05',
    'июня': '06',
    'июля': '07',
    'августа': '08',
    'сентября': '09',
    'октября': '10',
    'ноября': '11',
    'декабря': '12',
}


def int_value_from_ru_month(date_str):
    for k, v in RU_MONTH_VALUES.items():
        date_str = date_str.replace(k, str(v))
    return date_str

In [3]:
class SpidyQuotesSpider(scrapy.Spider):
    name = 'spidyquotes'
    quotes_base_url = 'https://минобрнауки.рф/пресс-центр/by-page?page=%s&events_sections=1'
    start_urls = [quotes_base_url % 1]
    download_delay = 1.5
    next_page = 1
    def parse(self, response):  
        html = response.text
        soup = BeautifulSoup(html, 'lxml')
        news = soup.find_all('div', class_='news-item')
        for new in news:
            url = 'https://минобрнауки.рф' + new.find('a', class_='media-item-link').get('href')
            name = new.find('a', class_='media-item-link').getText()
            date_str = new.find('span', class_ = 'published published-alt').getText().split(',')[0][:-5] 
            
            days = timedelta(20)
            deadline = datetime.now()-days
            
            date = int_value_from_ru_month(date_str.lower())
            date = datetime.strptime(date, '%d %m %Y')
            if date > deadline:
                yield {
                    'name': name,
                    'url': url,
                    'date': date_str,
                }
            else:
                break
                

        if date > deadline:
            if self.next_page < 307: 
                print(self.next_page)
                self.next_page = self.next_page + 1
                yield scrapy.Request('https://минобрнауки.рф/пресс-центр/by-page?page=%s&events_sections=1' % self.next_page)

In [4]:
if __name__ == "__main__":
    process = CrawlerProcess()

    process.crawl(SpidyQuotesSpider)
    process.start() # the script will block here until the crawling is finished
    process.join()

2018-07-03 04:15:35 [scrapy.utils.log] INFO: Scrapy 1.5.0 started (bot: scrapybot)
2018-07-03 04:15:35 [scrapy.utils.log] INFO: Versions: lxml 4.2.3.0, libxml2 2.9.8, cssselect 1.0.3, parsel 1.4.0, w3lib 1.19.0, Twisted 18.4.0, Python 3.6.4 | packaged by conda-forge | (default, Dec 23 2017, 16:31:06) - [GCC 4.8.2 20140120 (Red Hat 4.8.2-15)], pyOpenSSL 17.2.0 (OpenSSL 1.0.2n  7 Dec 2017), cryptography 2.0.3, Platform Linux-4.14.22+-x86_64-with-debian-stretch-sid
2018-07-03 04:15:35 [scrapy.crawler] INFO: Overridden settings: {}
2018-07-03 04:15:35 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2018-07-03 04:15:35 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloade

1


2018-07-03 04:15:37 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://xn--80abucjiibhv9a.xn--p1ai/%D0%BF%D1%80%D0%B5%D1%81%D1%81-%D1%86%D0%B5%D0%BD%D1%82%D1%80/by-page?page=2&events_sections=1> (referer: https://xn--80abucjiibhv9a.xn--p1ai/%D0%BF%D1%80%D0%B5%D1%81%D1%81-%D1%86%D0%B5%D0%BD%D1%82%D1%80/by-page?page=1&events_sections=1)
2018-07-03 04:15:38 [scrapy.core.scraper] DEBUG: Scraped from <200 https://xn--80abucjiibhv9a.xn--p1ai/%D0%BF%D1%80%D0%B5%D1%81%D1%81-%D1%86%D0%B5%D0%BD%D1%82%D1%80/by-page?page=2&events_sections=1>
{'name': 'Глава государства вручил награды за значимый вклад в развитие России', 'url': 'https://минобрнауки.рф/%D0%BF%D1%80%D0%B5%D1%81%D1%81-%D1%86%D0%B5%D0%BD%D1%82%D1%80/13086', 'date': '27 июня 2018'}
2018-07-03 04:15:38 [scrapy.core.scraper] DEBUG: Scraped from <200 https://xn--80abucjiibhv9a.xn--p1ai/%D0%BF%D1%80%D0%B5%D1%81%D1%81-%D1%86%D0%B5%D0%BD%D1%82%D1%80/by-page?page=2&events_sections=1>
{'name': 'Первые педагоги по шахматам прошли обучение в

2


2018-07-03 04:15:39 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://xn--80abucjiibhv9a.xn--p1ai/%D0%BF%D1%80%D0%B5%D1%81%D1%81-%D1%86%D0%B5%D0%BD%D1%82%D1%80/by-page?page=3&events_sections=1> (referer: https://xn--80abucjiibhv9a.xn--p1ai/%D0%BF%D1%80%D0%B5%D1%81%D1%81-%D1%86%D0%B5%D0%BD%D1%82%D1%80/by-page?page=2&events_sections=1)
2018-07-03 04:15:39 [scrapy.core.scraper] DEBUG: Scraped from <200 https://xn--80abucjiibhv9a.xn--p1ai/%D0%BF%D1%80%D0%B5%D1%81%D1%81-%D1%86%D0%B5%D0%BD%D1%82%D1%80/by-page?page=3&events_sections=1>
{'name': 'Российские выпускники создадут фотогалерею в социальной сети «ВКонтакте»', 'url': 'https://минобрнауки.рф/%D0%BF%D1%80%D0%B5%D1%81%D1%81-%D1%86%D0%B5%D0%BD%D1%82%D1%80/13071', 'date': '22 июня 2018'}
2018-07-03 04:15:39 [scrapy.core.scraper] DEBUG: Scraped from <200 https://xn--80abucjiibhv9a.xn--p1ai/%D0%BF%D1%80%D0%B5%D1%81%D1%81-%D1%86%D0%B5%D0%BD%D1%82%D1%80/by-page?page=3&events_sections=1>
{'name': 'В День памяти и скорби глава государства 

3


2018-07-03 04:15:41 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://xn--80abucjiibhv9a.xn--p1ai/%D0%BF%D1%80%D0%B5%D1%81%D1%81-%D1%86%D0%B5%D0%BD%D1%82%D1%80/by-page?page=4&events_sections=1> (referer: https://xn--80abucjiibhv9a.xn--p1ai/%D0%BF%D1%80%D0%B5%D1%81%D1%81-%D1%86%D0%B5%D0%BD%D1%82%D1%80/by-page?page=3&events_sections=1)
2018-07-03 04:15:41 [scrapy.core.scraper] DEBUG: Scraped from <200 https://xn--80abucjiibhv9a.xn--p1ai/%D0%BF%D1%80%D0%B5%D1%81%D1%81-%D1%86%D0%B5%D0%BD%D1%82%D1%80/by-page?page=4&events_sections=1>
{'name': 'С регионами обсудили ключевые направления работы в сфере образования', 'url': 'https://минобрнауки.рф/%D0%BF%D1%80%D0%B5%D1%81%D1%81-%D1%86%D0%B5%D0%BD%D1%82%D1%80/13051', 'date': '19 июня 2018'}
2018-07-03 04:15:41 [scrapy.core.scraper] DEBUG: Scraped from <200 https://xn--80abucjiibhv9a.xn--p1ai/%D0%BF%D1%80%D0%B5%D1%81%D1%81-%D1%86%D0%B5%D0%BD%D1%82%D1%80/by-page?page=4&events_sections=1>
{'name': '20 июня выпускники российских школ сдадут ЕГЭ

4


2018-07-03 04:15:43 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://xn--80abucjiibhv9a.xn--p1ai/%D0%BF%D1%80%D0%B5%D1%81%D1%81-%D1%86%D0%B5%D0%BD%D1%82%D1%80/by-page?page=5&events_sections=1> (referer: https://xn--80abucjiibhv9a.xn--p1ai/%D0%BF%D1%80%D0%B5%D1%81%D1%81-%D1%86%D0%B5%D0%BD%D1%82%D1%80/by-page?page=4&events_sections=1)
2018-07-03 04:15:43 [scrapy.core.scraper] DEBUG: Scraped from <200 https://xn--80abucjiibhv9a.xn--p1ai/%D0%BF%D1%80%D0%B5%D1%81%D1%81-%D1%86%D0%B5%D0%BD%D1%82%D1%80/by-page?page=5&events_sections=1>
{'name': 'Подведены итоги Всероссийской интернет-олимпиады для школьников на знание правил дорожного движения для младшей возрастной категории', 'url': 'https://минобрнауки.рф/%D0%BF%D1%80%D0%B5%D1%81%D1%81-%D1%86%D0%B5%D0%BD%D1%82%D1%80/13037', 'date': '14 июня 2018'}
2018-07-03 04:15:43 [scrapy.core.scraper] DEBUG: Scraped from <200 https://xn--80abucjiibhv9a.xn--p1ai/%D0%BF%D1%80%D0%B5%D1%81%D1%81-%D1%86%D0%B5%D0%BD%D1%82%D1%80/by-page?page=5&events_sec